In [11]:
import copy
from elasticsearch import Elasticsearch

ES_HOST = "https://cluster.elasticsearch.dataesr.ovh/"
ES_INDEX = "bso-publications-20230403"
ES_PASSWORD = "vn84q9Xef9U7pmU"
ES_USER = "BSO"

es = Elasticsearch(ES_HOST, http_auth=(ES_USER, ES_PASSWORD))

### All publications in ES
With foreign publications, publications from HAL ...

In [12]:
count_es = es.count(index=ES_INDEX).get("count")
count_es

/tmp/ipykernel_99371/1825937803.py:1: ElasticsearchWarning: The client is unable to verify that the server is Elasticsearch due security privileges on the server side
  count_es = es.count(index=ES_INDEX).get("count")


2961498

### Publications in the Monitor
Publications from France, with a DOI from crossref and of type "journal-article", "proceedings", "book-chapter", "book" or "preprint"

In [13]:
body = {
	"query": {
		"bool": {
			"must": [
				{
					"term": {
						"bso_country.keyword": "fr"
					}
				},
				{
					"terms": {
						"id_type.keyword": [
							"doi"
						]
					}
				},
				{
					"terms": {
						"genre.keyword": [
							"journal-article",
							"proceedings",
							"book-chapter",
							"book",
							"preprint"
						]
					}
				}
			]
		}
	}
}

count_bso = es.count(index=ES_INDEX, body=body).get("count")
count_bso

1593783

### Publications from the Monitor and analyzed by Datastet

In [14]:
body_copy = copy.deepcopy(body)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
count_bso_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_bso_datastet

656207

### Publications from the Monitor and analyzed by Softcite

In [15]:
body_copy = copy.deepcopy(body)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_bso_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_bso_datastet

657693

### Publications from the Monitor and analyzed by DataStet AND Softcite

In [16]:
body_copy = copy.deepcopy(body)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_bso_datastet_softcite = es.count(index=ES_INDEX, body=body_copy).get("count")
count_bso_datastet_softcite

655280

### Publications in ES and analyzed by DataStet

In [17]:
body2 = {
	"query": {
		"bool": {
			"must": [
			]
		}
	}
}

body_copy = copy.deepcopy(body2)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
count_es_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_es_datastet

710633

### Publications in ES and analyzed by Softcite

In [18]:
body_copy = copy.deepcopy(body2)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_es_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_es_datastet

712289

### Publications in ES and analyzed by DataStet AND Softcite

In [19]:
body_copy = copy.deepcopy(body2)
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "datastet_details" } })
body_copy.get("query").get("bool").get("must").append({ "exists": { "field": "softcite_details" } })
count_es_datastet = es.count(index=ES_INDEX, body=body_copy).get("count")
count_es_datastet

709620